# CubeSat attitude control - TEST FILE

This test file is meant to test the classes and other lines of code used for the thesis research. It will therefore contain Python windows with smaller unit tests and their results. Verification methods are also tested throughout this file. Packages are imported independently and constants will be retrieved from the Constants class.

In [1]:
import numpy as np
import scipy as sc
from scipy.integrate import RK45
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from astroquery.jplhorizons import Horizons
import datetime
import sys
import control as c
import serial as serial
import sliplib as slip
import classes as cl
from astropy.time import Time
from scipy.linalg import null_space

# %run ./simulation.ipynb

### Paper examples


Paper in question is "Robust Optimal Solution to the Attitude/Force Control Problem" from Peña and Alonso. The method, stated in Theorem 1, was adapted in the paper "Attitude Control for the LUMIO CubeSat in Deep Space".

To do: Verify that you indeed take two nullspace vectors in case of redundant thrusters. Also check if it works for more than 4 thrusters.

In [13]:
def thruster_ax(theta):
    return np.array([np.sin(theta), 0, np.cos(theta)])

x1, x2, x3, x4 = 0.5, 1, -1, 1
y1, y2, y3, y4 = -1, -1, 1, 1
z1, z2, z3, z4 = -1, -1, -1, -1
th1, th2, th3, th4 = np.deg2rad(10), np.deg2rad(-10), np.deg2rad(10), np.deg2rad(-10)

d_1 = np.array([x1, y1, z1])
d_2 = np.array([x2, y2, z2])
d_3 = np.array([x3, y3, z3])
d_4 = np.array([x4, y4, z4])

e_1 = thruster_axes(th1)
e_2 = thruster_axes(th2)
e_3 = thruster_axes(th3)
e_4 = thruster_axes(th4)

A = np.column_stack((np.cross(d_1, e_1), np.cross(d_2, e_2), np.cross(d_3, e_3), np.cross(d_4, e_4)))

A_test = np.array([[y1 * np.cos(th1), (z1*np.sin(th1) - x1*np.cos(th1)), -y1*np.sin(th1)], [y2 * np.cos(th2), (z2*np.sin(th2) - x2*np.cos(th2)), -y2*np.sin(th2)], [y3 * np.cos(th3), (z3*np.sin(th3) - x3*np.cos(th3)), -y3*np.sin(th3)], [y4 * np.cos(th4), (z4*np.sin(th4) - x4*np.cos(th4)), -y4*np.sin(th4)]]).T

kern_A = null_space(A)

pseudo_A = np.linalg.pinv(A)

torque = np.array([-0.1, 0, 0])

a  = -pseudo_A @ torque
gamma_list = []

for i in range(len(a)):
    gamma_list.append(a[i] / kern_A[i])

gamma = max(gamma_list)
F = pseudo_A @ torque + (gamma * kern_A.T)

print(F)

[[0.56762945 0.05077133 0.51685812 0.        ]]


### New setup verification

Adjust the angle definitions from the paper, create A-matrix, check results

In [3]:
def thruster_axes(phi, lam):
    return np.array([np.cos(lam)*np.cos(np.pi / 2 - phi), np.sin(lam), np.cos(lam)*np.cos(phi)])

x1, x2, x3, x4 = 0.5, 1, -1, 1
y1, y2, y3, y4 = -1, -1, 1, 1
z1, z2, z3, z4 = -1, -1, -1, -1

d_1 = np.array([x1, y1, z1])
d_2 = np.array([x2, y2, z2])
d_3 = np.array([x3, y3, z3])
d_4 = np.array([x4, y4, z4])

phi1, phi2, phi3, phi4 = np.deg2rad(10), np.deg2rad(-10), np.deg2rad(10), np.deg2rad(-10)
lam1, lam2, lam3, lam4 = 0, 0, 0, 0

e_1 = thruster_axes(phi1, lam1)
e_2 = thruster_axes(phi2, lam2)
e_3 = thruster_axes(phi3, lam3)
e_4 = thruster_axes(phi4, lam4)

A = np.column_stack((np.cross(d_1, e_1), np.cross(d_2, e_2), np.cross(d_3, e_3), np.cross(d_4, e_4)))

kern_A = null_space(A)

pseudo_A = np.linalg.pinv(A)

torque = np.array([-0.1, 0, 0])

a  = -pseudo_A @ torque
gamma_list = []

for i in range(len(a)):
    gamma_list.append(a[i] / kern_A[i])

gamma = max(gamma_list)
F = pseudo_A @ torque + (gamma * kern_A.T)

print(kern_A)
print(F)

[[0.7042951 ]
 [0.06299532]
 [0.7042951 ]
 [0.06299532]]
[[0.56762945 0.05077133 0.51685812 0.        ]]


### Thesis version

First step: see if we can adjust it to the LUMIO thruster set-up from the ADCS descriptive papers.

LUMIO setup worked correctly.

Addition: one extra thruster on one of the corners of the base plate.

In [46]:
# Thruster data
def thruster_axes(phi, lam):
    return np.array([np.cos(lam)*np.cos(np.pi / 2 - phi), np.sin(lam), np.cos(lam)*np.cos(phi)])

c = (np.sqrt(10**2 + 10**2)-10) / 2 * np.cos(np.pi / 4) / 100 # in m

x_L_1, x_L_2, x_L_3, x_L_4 =  c, -0.1+c, -c, 0.1 - c
y_L_1, y_L_2, y_L_3, y_L_4 = -0.15, -0.15, -0.15, -0.15
z_L_1, z_L_2, z_L_3, z_L_4 = -0.1+c, c, 0.1 - c, -c

x_L_5, y_L_5, z_L_5 = -0.1, -0.15, -0.1 # Corner of the plate

phi_L_1, phi_L_2, phi_L_3, phi_L_4 = np.pi / 2, 0, 3/2*np.pi, np.pi
lam_L_1, lam_L_2, lam_L_3, lam_L_4 = np.deg2rad(110), np.deg2rad(80), np.deg2rad(80), np.deg2rad(80)

phi_L_5, lam_L_5 = np.deg2rad(250), np.deg2rad(80) # same cant angle, slightly pointed inwards

d_L_1 = np.array([x_L_1, y_L_1, z_L_1])
d_L_2 = np.array([x_L_2, y_L_2, z_L_2])
d_L_3 = np.array([x_L_3, y_L_3, z_L_3])
d_L_4 = np.array([x_L_4, y_L_4, z_L_4])
d_L_5 = np.array([x_L_5, y_L_5, z_L_5])

e_L_1 = thruster_axes(phi_L_1, lam_L_1)
e_L_2 = thruster_axes(phi_L_2, lam_L_2)
e_L_3 = thruster_axes(phi_L_3, lam_L_3)
e_L_4 = thruster_axes(phi_L_4, lam_L_4)
e_L_5 = thruster_axes(phi_L_5, lam_L_5)

A_L = np.column_stack((np.cross(d_L_1, e_L_1), np.cross(d_L_2, e_L_2), np.cross(d_L_3, e_L_3), np.cross(d_L_4, e_L_4), np.cross(d_L_5, e_L_5)))

kern_A_L = null_space(A_L)
print(kern_A_L)
pseudo_A_L = np.linalg.pinv(A_L)
torqu = np.array([0.01, 0.005, 0.002])
a_L  = -pseudo_A_L @ torqu

if kern_A_L.shape[1] > 1:
    for j in range(kern_A_L.shape[1]):
        gamma_list = []
        for i in range(len(a_L)):
            print(kern_A_L.T[j][i])
            gamma_list.append(a_L[i] / kern_A_L.T[j][i])
        gamma_L = max(gamma_list)
        F_LUMIO = pseudo_A_L @ torqu + (gamma_L * kern_A_L.T[j])
        print(F_LUMIO)

else:
    gamma_list = []
    gamma_L = max(gamma_list)
    F_LUMIO = pseudo_A_L @ torqu + (gamma_L * kern_A_L.T)

    # print(pseudo_A_L)
    # print(kern_A_L)
    # print(F_LUMIO)


[[-0.1682142  -0.66539199]
 [-0.00083796  0.46218829]
 [ 0.60477546 -0.51813553]
 [ 0.70328567  0.19599906]
 [ 0.33367527  0.19171704]]
[-0.09017697 -0.0542435   0.04717507 -0.07038028  0.01724016]
-0.1682142026148788
-0.0008379550468383147
0.6047754638490144
0.7032856674317921
0.3336752735538076
[-10.79886532   0.          39.10187119  45.59630647  21.58262542]
-0.6653919916536404
0.4621882873858297
-0.518135531859371
0.1959990619867804
0.19171703829254338
[ 0.          0.11688138 -0.11739517  0.09694297  0.00874221]


### Test generalized code

Works for 6 thrusters, more to be tested

In [25]:
def thruster_axes(phi, lam):
    return np.array([np.cos(lam) * np.cos(np.pi / 2 - phi), np.sin(lam), np.cos(lam) * np.cos(phi)])

def thruster_configuration(array_thrusters):
    num_thrusters = len(array_thrusters)
    d = np.zeros((3, num_thrusters))
    e = np.zeros((3, num_thrusters))

    for i in range(num_thrusters):
        x, y, z, phi, lam = array_thrusters[i]
        d[:, i] = [x, y, z]
        e[:, i] = thruster_axes(np.deg2rad(phi), np.deg2rad(lam))

    A_L = np.column_stack([np.cross(d[:, i], e[:, i]) for i in range(num_thrusters)])
    kern_A_L = null_space(A_L)

    # Error handling for the kernel
    valid_kernels = [kern_A_L[:, i] for i in range(kern_A_L.shape[1]) if np.all(kern_A_L[:, i] > 0)]

    if not valid_kernels:
        print("No valid kernel found. All kernels have non-positive elements.")
        return

    # Compute control actions if valid kernels exist
    torqu = np.array([0.001, 0.00003, 0.002])
    pseudo_A_L = np.linalg.pinv(A_L)
    a_L = -pseudo_A_L @ torqu

    min_force = np.inf
    best_F_LUMIO = None

    for j in range(len(valid_kernels)):
        gamma_list = [a_L[i] / valid_kernels[j][i] for i in range(len(a_L))]
        gamma_L = max(gamma_list)
        F_LUMIO = pseudo_A_L @ torqu + (gamma_L * valid_kernels[j])
        force_magnitude = np.linalg.norm(F_LUMIO)

        if force_magnitude < min_force:
            min_force = force_magnitude
            best_F_LUMIO = F_LUMIO

    print("Minimum force vector:", best_F_LUMIO)
    print("Minimum force magnitude:", min_force)

# Example usage
array_thrusters = [
    [0.5, -1, -1, 10, 0],
    [1, -1, -1, -10, 0],
    [-1, 1, -1, 10, 0],
    [1, 1, -1, -10, 0],
    [-1,-1,1,30,0],
    [1,1,1,30,0]
]


thruster_configuration(array_thrusters)

Minimum force vector: [0.00977275 0.01131188 0.01787757 0.01047185 0.00710651 0.        ]
Minimum force magnitude: 0.026518577163725524


### With thrust limits

In [3]:
def thruster_configuration_lim(array_thrusters, lim_value):
    num_thrusters = len(array_thrusters)
    thrust_limit = np.ones(num_thrusters) * lim_value # Assuming equal limits for each thruster
    d = np.zeros((3, num_thrusters))
    e = np.zeros((3, num_thrusters))

    for i in range(num_thrusters):
        x, y, z, phi, lam = array_thrusters[i]
        d[:, i] = [x, y, z]
        e[:, i] = thruster_axes(np.deg2rad(phi), np.deg2rad(lam))

    A_L = np.column_stack([np.cross(d[:, i], e[:, i]) for i in range(num_thrusters)])
    kern_A_L = null_space(A_L)

    # Error handling for the kernel
    valid_kernels = [kern_A_L[:, i] for i in range(kern_A_L.shape[1]) if np.all(kern_A_L[:, i] > 0)]

    if not valid_kernels:
        print("No valid kernel found. All kernels have non-positive elements.")
        return

    # Compute control actions if valid kernels exist
    torqu = np.array([0.001, 0.00003, 0.002])
    pseudo_A_L = np.linalg.pinv(A_L)
    a_L = -pseudo_A_L @ torqu

    min_force = np.inf
    best_F_LUMIO = None

    for j in range(len(valid_kernels)):
        gamma_list = [thrust_limit[i] - a_L[i] / valid_kernels[j][i] for i in range(len(a_L))]
        gamma_L = min(gamma_list)
        F_LUMIO = pseudo_A_L @ torqu + (gamma_L * valid_kernels[j])
        force_magnitude = np.linalg.norm(F_LUMIO)

        if force_magnitude < min_force:
            min_force = force_magnitude
            best_F_LUMIO = F_LUMIO

    if np.any(best_F_LUMIO < 0):
        print("Error: Negative thrust instance detected in the optimal force vector.")
    else:
        print("Minimum force vector:", best_F_LUMIO)
        print("Minimum force magnitude:", min_force)

# Example usage
array_thrusters = [
    [0.5, -1, -1, 10, 0, 0.1],
    [1, -1, -1, -10, 0, 0.1],
    [-1, 1, -1, 10, 0, 0.1],
    [1, 1, -1, -10, 0, 0.1],
    [-1,-1,1,30,0, 0.1],
    [1,1,1,30,0, 0.1],
]


cont = np.array([0.1, 0.003, 0.005])
lim = np.array([0.1, 0.002, 0.3])
k4 = 0.8
thrusters = cl.thruster_allocation(cont, lim, k4)

T = thrusters.thruster_configuration(array_thrusters)




ValueError: too many values to unpack (expected 5)

In [55]:
array_thrusters = [
    [0.5, -1, -1, 10, 0,5],
    [1, -1, -1, -10, 0,3],
    [-1, 1, -1, 10, 0,2],
    [1, 1, -1, -10, 0,1],
    [-1,-1,1,30,0,3],
    [1,1,1,30,0,4],
]

print(array_thrusters[1][0])

t = np.ones(5)
lim = np.array([1,2,3,4,5])

num_thrusters = len(array_thrusters)
lim_values = [array_thrusters[i][-1] for i in range(len(array_thrusters))]
thrust_limit = np.ones(num_thrusters) * lim_values # Assuming equal limits for each thruster

print(thrust_limit)

a = 5
b = 6
c = 5

if a => b:
    print(c)


SyntaxError: invalid syntax (778360473.py, line 25)

In [17]:
S_loc_1 = np.array([0, 0, 0.1])
S_loc_2 = np.array([0, 0, -0.1])
S_loc_3 = np.array([-0.1, 0, 0])
S_loc_4 = np.array([0.1, 0, 0])
S_loc_5 = np.array([0, -0.1, 0])
S_loc_6 = np.array([0, 0.1, 0])

position_Sun_Moon = np.array([1000, 10000, 1000])
position_SC_Moon = np.array([10, 100, 10])
c_p = np.column_stack([S_loc_1, S_loc_2, S_loc_3, S_loc_4, S_loc_5, S_loc_6]) # Centre of pressure locations for calculation
n_s = np.array([[0,0,1,-1,0,0], [0,0,0,0,1,-1],[-1,1,0,0,0,0]])
r_S_SC = position_Sun_Moon - position_SC_Moon
S = np.column_stack([(S_loc_1 - r_S_SC) / np.linalg.norm(S_loc_1 - r_S_SC), (S_loc_2 - r_S_SC) / np.linalg.norm(S_loc_2 - r_S_SC), (S_loc_3 - r_S_SC) / np.linalg.norm(S_loc_3 - r_S_SC), (S_loc_4 - r_S_SC) / np.linalg.norm(S_loc_4 - r_S_SC), (S_loc_5 - r_S_SC) / np.linalg.norm(S_loc_5 - r_S_SC), (S_loc_6 - r_S_SC) / np.linalg.norm(S_loc_6 - r_S_SC)])

print(S)

print(S.T[0])


[[-0.09901485 -0.09901466 -0.09902466 -0.09900485 -0.09901377 -0.09901573]
 [-0.99014852 -0.99014656 -0.99014656 -0.99014852 -0.99014774 -0.99014735]
 [-0.09900485 -0.09902466 -0.09901466 -0.09901485 -0.09901377 -0.09901573]]
[-0.09901485 -0.99014852 -0.09900485]


In [23]:
control_torque = [1,2,3]
lower_limit_torque = [5, 1, 1]
k4 = 0.1

cross = np.cross(control_torque, lower_limit_torque)

test = np.empty(3)

test[0], test[1], test[2] = np.zeros(3)

print(test)
print(cross)

array = np.array([
    [1, 2, 3, 4, 5, 6],
    [7, 8, 9, 10, 11, 12],
    [13, 14, 15, 16, 17, 18]
])

# Summing across the columns to reduce each row to a single sum value
row_sums = array.sum(axis=1)

print(row_sums)

[0. 0. 0.]
[-1 14 -9]
[21 57 93]
